In [22]:
%load_ext autoreload
%autoreload 2


from dataclasses import dataclass
from decimal import Decimal
from typing import Optional

import polars as pl
from op_analytics.coreutils.bigquery.write import (
    most_recent_dates,
)
from op_analytics.coreutils.logger import structlog
from op_analytics.coreutils.request import get_data, new_session
from op_analytics.coreutils.threads import run_concurrently
from op_analytics.coreutils.time import dt_fromepoch, now_dt

from op_analytics.datasources.defillama.dataaccess import DefiLlama

log = structlog.get_logger()

SUMMARY_ENDPOINT = "https://stablecoins.llama.fi/stablecoins?includePrices=true"
BREAKDOWN_ENDPOINT = "https://stablecoins.llama.fi/stablecoin/{id}"


BALANCES_TABLE_LAST_N_DAYS = 7  # upsert only the last 7 days of balances fetched from the api


MUST_HAVE_METADATA_FIELDS = [
    "id",
    "name",
    "address",
    "symbol",
    "url",
    "pegType",
    "pegMechanism",
]

OPTIONAL_METADATA_FIELDS = [
    "description",
    "mintRedeemDescription",
    "onCoinGecko",
    "gecko_id",
    "cmcId",
    "priceSource",
    "twitter",
    "price",
]

METADATA_DF_SCHEMA = {
    "id": pl.String,
    "name": pl.String,
    "address": pl.String,
    "symbol": pl.String,
    "url": pl.String,
    "pegType": pl.String,
    "pegMechanism": pl.String,
    "description": pl.String,
    "mintRedeemDescription": pl.String,
    "onCoinGecko": pl.String,
    "gecko_id": pl.String,
    "cmcId": pl.String,
    "priceSource": pl.String,
    "twitter": pl.String,
    "price": pl.Float64,
}

BALANCES_DF_SCHEMA = {
    "id": pl.String(),
    "chain": pl.String(),
    "dt": pl.String(),
    "circulating": pl.Decimal(scale=18),
    "bridged_to": pl.Decimal(scale=18),
    "minted": pl.Decimal(scale=18),
    "unreleased": pl.Decimal(scale=18),
    "name": pl.String(),
    "symbol": pl.String(),
}


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:

@dataclass
class DefillamaStablecoins:
    """Metadata and balances for all stablecoins.

    This is the result we obtain after fetching from the API and extracting the data
    that we need to ingest.
    """

    metadata_df: pl.DataFrame
    balances_df: pl.DataFrame


def construct_urls(stablecoins_summary, symbols: list[str] | None) -> dict[str, str]:
    """Build the collection of urls that we will fetch from DefiLlama.

    Args:
        symbols: list of symbols to process. Defaults to None (process all).
    """
    urls = {}
    for stablecoin in stablecoins_summary:
        stablecoin_symbol = stablecoin["symbol"]
        stablecoin_id = stablecoin["id"]
        if symbols is None or stablecoin_symbol in symbols:
            urls[stablecoin_id] = BREAKDOWN_ENDPOINT.format(id=stablecoin_id)

    if not urls:
        raise ValueError("No valid stablecoin IDs provided.")
    return urls


def pull_stablecoins(symbols: list[str] | None = None) -> DefillamaStablecoins:
    """
    Pulls and processes stablecoin data from DeFiLlama.

    Args:
        stablecoin_ids: list of stablecoin IDs to process. Defaults to None (process all).
    """
    session = new_session()

    # Call the summary endpoint to find the list of stablecoins tracked by DefiLLama.
    summary = get_data(session, SUMMARY_ENDPOINT)
    stablecoins_summary = summary["peggedAssets"]

    # Call the API endpoint for each stablecoin in parallel.
    urls = construct_urls(stablecoins_summary, symbols=symbols)
    stablecoins_data = run_concurrently(lambda x: get_data(session, x), urls, max_workers=4)

    # Extract all the balances (includes metadata).
    result = extract(stablecoins_data)

    # Write metadata.
    DefiLlama.STABLECOINS_METADATA.write(
        dataframe=result.metadata_df.with_columns(dt=pl.lit(now_dt())),
        sort_by=["symbol"],
    )

    # Write balances.
    DefiLlama.STABLECOINS_BALANCE.write(
        dataframe=most_recent_dates(result.balances_df, n_dates=BALANCES_TABLE_LAST_N_DAYS),
        sort_by=["symbol", "chain"],
    )

    return result


def execute_pull():
    result = pull_stablecoins()
    return {
        "metadata_df": len(result.metadata_df),
        "balances_df": len(result.balances_df),
    }


def extract(stablecoins_data) -> DefillamaStablecoins:
    """Extract metadata and balances for all stablecoins."""
    metadata: list[dict] = []
    balances: list[dict] = []
    for stablecoin_id, data in stablecoins_data.items():
        assert stablecoin_id == data["id"]
        metadata_row, balance_rows = single_stablecoin_balances(data)

        if not metadata_row:
            raise ValueError(f"No metadata for stablecoin={data['name']}")

        if not balance_rows:
            raise ValueError(f"No balances for stablecoin={data['name']}")

        metadata.append(metadata_row)
        balances.extend(balance_rows)

    metadata_df = pl.DataFrame(metadata, infer_schema_length=len(metadata))
    balances_df = pl.DataFrame(balances, schema=BALANCES_DF_SCHEMA)

    # Schema assertions to help our future selves reading this code.
    assert metadata_df.schema == METADATA_DF_SCHEMA
    assert balances_df.schema == BALANCES_DF_SCHEMA

    return DefillamaStablecoins(metadata_df=metadata_df, balances_df=balances_df)


def single_stablecoin_metadata(data: dict) -> dict:
    """Extract metadata for a single stablecoin.

    Will fail if the response data from the API is missing any of the "must-have"
    metadata fields.

    Args:
        data: Data for this stablecoin as returned by the API

    Returns:
        The metadata dictionary.
    """
    metadata: dict[str, Optional[str]] = {}

    # Collect required metadata fields
    for key in MUST_HAVE_METADATA_FIELDS:
        metadata[key] = data[key]

    # Collect additional optional metadata fields
    for key in OPTIONAL_METADATA_FIELDS:
        metadata[key] = data.get(key)

    return metadata


def safe_decimal(float_val):
    """Safely convert DefiLLama balance values to Decimal.

    Balance values can be int or float. This function converts values to Decimal,
    so we don't have to rely on polars schema inference.
    """
    if float_val is None:
        return None

    return Decimal(str(float_val))


def single_stablecoin_balances(data: dict) -> tuple[dict, list[dict]]:
    """Extract balances for a single stablecoin.

    Args:
        data: Data for this stablecoin as returned by the API

    Returns:
        Tuple of metadata dict and balances for this stablecoin.
        Each item in balances is one data point obtained from DefiLlama.
    """
    metadata = single_stablecoin_metadata(data)
    peg_type: str = data["pegType"]

    def get_value(_datapoint, _metric_name):
        """Helper to get a nested dict key with fallback."""
        return _datapoint.get(_metric_name, {}).get(peg_type)

    balances = []

    for chain, balance in data["chainBalances"].items():
        tokens = balance.get("tokens", [])

        for datapoint in tokens:
            row = {
                "id": data["id"],
                "chain": chain,
                "dt": dt_fromepoch(datapoint["date"]),
                "circulating": safe_decimal(get_value(datapoint, "circulating")),
                "bridged_to": safe_decimal(get_value(datapoint, "bridgedTo")),
                "minted": safe_decimal(get_value(datapoint, "minted")),
                "unreleased": safe_decimal(get_value(datapoint, "unreleased")),
                "name": metadata["name"],
                "symbol": metadata["symbol"],
            }
            balances.append(row)

    return metadata, balances


In [8]:
result = pull_stablecoins()

2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoins?includePrices=true: 0.18 seconds filename=request.py lineno=81 process=55515
2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoin/3: 0.18 seconds counter=003/212 eta=None filename=request.py lineno=81 process=55515
2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoin/2: 0.21 seconds counter=002/212 eta=None filename=request.py lineno=81 process=55515
2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoin/4: 0.32 seconds counter=004/212 eta=None filename=request.py lineno=81 process=55515
2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoin/5: 0.21 seconds counter=005/212 eta=None filename=request.py lineno=81 process=55515
2025-01-17 12:54:02 [info     ] Fetched from https://stablecoins.llama.fi/stablecoin/6: 0.12 seconds counter=006/212 eta=None filename=request.py lineno=81 p

In [6]:
metadata_df

'metadata_df'

In [9]:
result.balances_df.to_pandas()

,id,chain,dt,circulating,bridged_to,minted,unreleased,name,symbol
0,3,Optimism,2022-06-03,100326.000000000000000000,100326.000000000000000000,None,None,TerraClassicUSD,USTC
1,3,Optimism,2022-06-04,100326.000000000000000000,100326.000000000000000000,None,None,TerraClassicUSD,USTC
2,3,Optimism,2022-06-05,100326.000000000000000000,100326.000000000000000000,None,None,TerraClassicUSD,USTC
3,3,Optimism,2022-06-06,100326.000000000000000000,100326.000000000000000000,None,None,TerraClassicUSD,USTC
4,3,Optimism,2022-06-07,100326.000000000000000000,100326.000000000000000000,None,None,TerraClassicUSD,USTC
...,...,...,...,...,...,...,...,...,...
480066,224,Fraxtal,2025-01-13,1652496.000000000000000000,None,1652496.000000000000000000,None,dTRINITY USD,dUSD
480067,224,Fraxtal,2025-01-14,1652496.000000000000000000,None,1652496.000000000000000000,None,dTRINITY USD,dUSD
480068,224,Fraxtal,2025-01-15,1652496.000000000000000000,None,1652496.000000000000000000,None,dTRINITY USD,dUSD
480069,224,Fraxtal,2025-01-16,1656943.000000000000000000,None,1656943.000000000000000000,None,dTRINITY USD,dUSD


In [24]:
from op_analytics.datasources.defillama.tvl_breakdown_enrichment import DefillamaTVLBreakdown


In [25]:
test = DefillamaTVLBreakdown.of_date("2025-01-01")

2025-01-17 13:09:00 [info     ] Reading data from 'defillama/protocols_metadata_v1' with filters min_date=2024-12-31, max_date=2025-01-01, date_range_spec=None filename=dailydata.py lineno=188 process=55515
2025-01-17 13:09:00 [info     ] querying markers for 'defillama/protocols_metadata_v1' DateFilter(min_date=datetime.date(2024, 12, 31), max_date=datetime.date(2025, 1, 1), datevals=None) filename=dailydata.py lineno=105 process=55515
2025-01-17 13:09:01 [info     ] 1 markers found                filename=dailydata.py lineno=119 max_dt=2024-12-31 min_dt=2024-12-31 process=55515
2025-01-17 13:09:01 [info     ] 1 distinct paths               filename=dailydata.py lineno=129 process=55515
2025-01-17 13:09:02 [info     ] registered view: 'defillama_protocols_metadata_v1' using 1 parquet paths filename=client.py lineno=273 process=55515
┌──────────────────────────────────┐
│               name               │
│             varchar              │
├──────────────────────────────────┤
│ defi

In [26]:
test.df_tvl_breakdown.to_pandas()

,dt,protocol_slug,chain,token,app_token_tvl,app_token_tvl_usd,protocol_name,protocol_category,parent_protocol,misrepresented_tokens,is_protocol_misrepresented,is_double_counted,to_filter_out
0,2024-12-29,nolus-protocol,Osmosis,OSMO,5.972255e+05,2.786774e+05,Nolus Protocol,Lending,nolus-protocol,0,0,0,0
1,2024-12-26,uniswap-v3,Ethereum,CBETH,2.498653e+03,9.436688e+06,Uniswap V3,Dexes,uniswap,0,0,0,0
2,2024-12-29,axelar,Osmosis,LVN,6.808130e+02,9.397300e+00,Axelar,Bridge,axelar,0,0,1,0
3,2024-12-30,archly-v1,Optimism,WETH,1.067000e-02,3.622993e+01,Archly V1,Dexes,archly-finance,0,0,0,0
4,2024-12-27,alex,Stacks,WGIGGLE,5.142357e+09,3.298479e+03,ALEX,Dexes,alex,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
679905,2024-12-26,pangea-swap,Klaytn,SSX,1.000000e+01,3.720000e-03,Pangea Swap,Dexes,pangea-swap,0,0,0,0
679906,2024-12-27,betfolio,Polygon,USDC,5.753800e+03,5.759554e+03,BetFolio,Prediction Market,betfolio,0,0,0,0
679907,2024-12-29,curve-dex,Ethereum,SUSDX,4.757654e+05,4.988083e+05,Curve DEX,Dexes,curve-finance,0,0,0,0
679908,2024-12-28,balancer-v2,Polygon,BCT,7.357508e+01,2.902625e+01,Balancer V2,Dexes,balancer,0,0,0,0
